### Introduction

In [ ]:
# Aim: to produce the resources to facilitate research on gene pleiotropy

In [ ]:
# Deliverable #1: Produce lists of Gene Ontologies and their corrected semantic similarity measures

# step #1: download the latest gene ontology annotations from here (make note of the date of download):
# http://current.geneontology.org/products/pages/downloads.html
# when multiple options exist (for example in human), there apply the following rules:
# (a) no isoform,
# (b) yes protein,
# (c) the most abundant annotations
# I suggest to study 6-7 model species, namely: humans, mouse, zebrafish, chicken, C. elegans, 
# D. melanogaster, other? 

### Import the data

In [54]:
import pandas as pd

# info on column_labels at: http://geneontology.org/docs/go-annotation-file-gaf-format-2.0/
column_labels = ['DB', 'DB Object ID', 'DB Object Symbol',
                 'Qualifier', 'GO ID', 'DB:Reference (|DB:Reference)',
                 'Evidence Code', 'With (or) From', 'Aspect', 'DB Object Name',
                 'DB Object Synonym (|Synonym)', 'DB Object Type', 'Taxon(|taxon)',
                 'Date', 'Assigned By', 'Annotation Extension', 'Gene Product Form ID']

df = pd.read_csv('goa_human.gaf', comment='!', 
                 sep="\t", header=None, low_memory=False, names=column_labels)

In [55]:
df.head(5)

,DB,DB Object ID,DB Object Symbol,Qualifier,GO ID,DB:Reference (|DB:Reference),Evidence Code,With (or) From,Aspect,DB Object Name,DB Object Synonym (|Synonym),DB Object Type,Taxon(|taxon),Date,Assigned By,Annotation Extension,Gene Product Form ID
0,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0003723,GO_REF:0000043,IEA,UniProtKB-KW:KW-0694,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20230507,UniProt,NaN,NaN
1,UniProtKB,A0A024RBG1,NUDT4B,enables,GO:0046872,GO_REF:0000043,IEA,UniProtKB-KW:KW-0479,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20230507,UniProt,NaN,NaN
2,UniProtKB,A0A024RBG1,NUDT4B,located_in,GO:0005829,GO_REF:0000052,IDA,NaN,C,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20161204,HPA,NaN,NaN
3,UniProtKB,A0A075B6H7,IGKV3-7,involved_in,GO:0002250,GO_REF:0000043,IEA,UniProtKB-KW:KW-1064,P,Probable non-functional immunoglobulin kappa v...,IGKV3-7,protein,taxon:9606,20230507,UniProt,NaN,NaN
4,UniProtKB,A0A075B6H7,IGKV3-7,located_in,GO:0005886,GO_REF:0000044,IEA,UniProtKB-SubCell:SL-0039,C,Probable non-functional immunoglobulin kappa v...,IGKV3-7,protein,taxon:9606,20230507,UniProt,NaN,NaN


In [56]:
df.shape

(632318, 17)

### Filtering

In [57]:
# for each gene (column: DB Object ID), retrieve the gene ontology annotations, as follows:
# (a) separately for P (biological process), F (molecular function) or C (cellular component)
# (column: Aspect),
# (b) end lists should not have duplicates, (forgot about that one)
# (c) do not incluce "NOT" qualifiers

# Filter for rows with qualifiers not containing "NOT"
df_filtered = df[~df["Qualifier"].str.contains("NOT")]


In [58]:
df_filtered.shape

(631035, 17)

### Create the dictionaries and populate them

In [60]:
# Create dictionaries for each aspect (P, F, and C)
aspect_P = {}
aspect_F = {}
aspect_C = {}

In [61]:
# Iterate through the filtered DataFrame and populate the dictionaries
for index, row in df_filtered.iterrows():
    aspect = row["Aspect"]
    gene_id = row["DB Object ID"]
    go_id = row["GO ID"]

    if aspect == "P":
        aspect_P.setdefault(gene_id, []).append(go_id)
    elif aspect == "F":
        aspect_F.setdefault(gene_id, []).append(go_id)
    elif aspect == "C":
        aspect_C.setdefault(gene_id, []).append(go_id) 

In [62]:
# remove duplicates
# in the case of key A0A087WXM9, we see that there are duplicates in the list of values
aspect_P['A0A087WXM9']

['GO:0007060',
 'GO:0010789',
 'GO:0016321',
 'GO:0045143',
 'GO:0051754',
 'GO:0010789',
 'GO:0007060',
 'GO:0045143',
 'GO:0051754',
 'GO:0016321']

In [63]:
# lets delete the duplicates
def remove_duplicates_from_list(aspect_P):
    cleaned_data = {}

    for key, value in aspect_P.items():
        if isinstance(value, list):
            cleaned_data[key] = list(dict.fromkeys(value))
        else:
            cleaned_data[key] = value

    return cleaned_data

# Remove duplicates from the dataset
cleaned_data = remove_duplicates_from_list(aspect_P)

# Print the list of values of key A0A087WXM9 to see results (it works)
cleaned_data['A0A087WXM9']

['GO:0007060', 'GO:0010789', 'GO:0016321', 'GO:0045143', 'GO:0051754']

### Useful stuff about the dictionaries

In [75]:
# Get the size of each dict (genes in each dict)
print(f"The size of dictionary aspect_P is: {len(cleaned_data)}")

The size of dictionary aspect_P is: 17818


In [88]:
print(f"The size of dictionary aspect_F is: {len(aspect_F)}")

The size of dictionary aspect_F is: 18188


In [89]:
print(f"The size of dictionary aspect_C is: {len(aspect_C)}")

The size of dictionary aspect_C is: 18890


In [64]:
# Print the first three key-value pairs of the P dictionary as an example
for gene_id, go_ids in list(cleaned_data.items())[:10]:
    print(f"{gene_id}: {go_ids}")

A0A075B6H7: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6H8: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6H9: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6I0: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6I1: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6I3: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6I4: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6I6: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6I7: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6I9: ['GO:0002250', 'GO:0006955', 'GO:0002377']


In [65]:
# search if a key (a gene) exists in the P dictionary
if "A0A024RBG1" in cleaned_data:
    print("Key 'A0A024RBG1' exists in the dictionary.")
else:
    print("Key 'A0A024RBG1' does not exist in the dictionary.")

Key 'A0A024RBG1' exists in the dictionary.


In [66]:
# get the value of a specific key (the GO terms associated with a gene)
cleaned_data["A0A024RBG1"] 

['GO:1901907', 'GO:1901909', 'GO:0071543', 'GO:1901911']

In [67]:
# sort the aspect_P dictionary based on keys (alphabetically)
sorted_Pdict = dict(sorted(cleaned_data.items(), key=lambda item: item[0]))

# print the first 3 rows of the sorted Pdict
for gene_id, go_ids in list(sorted_Pdict.items())[:10]:
    print(f"{gene_id}: {go_ids}")

A0A024RBG1: ['GO:1901907', 'GO:1901909', 'GO:0071543', 'GO:1901911']
A0A075B6H5: ['GO:0007166']
A0A075B6H7: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6H8: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6H9: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6I0: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6I1: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6I3: ['GO:0002250', 'GO:0002377', 'GO:0006955']
A0A075B6I4: ['GO:0002250', 'GO:0006955', 'GO:0002377']
A0A075B6I6: ['GO:0002250', 'GO:0006955', 'GO:0002377']


### Semantic similarity correction

In [ ]:
# now we will focus only on the "P" dictionary (biological process) and we will try to correct 
# by a semantic similarity measure the list of Gene Ontologies associated with each gene
# !!!! I suppose we want to set a semantic similarity threshold, where:
# !!!! if two terms are way too similar 
# !!!! (over the threshold, with 1 being identical and 0 being not similar)
# !!!! we keep only one of the two terms 
# !!!! (which one do we keep though? the more general (lower level) or the more specific (higher level))
# !!!! revigo example

# To correct the list of GO IDs associated with each gene ID using a semantic similarity measure, 
# we can use various approaches based on the Gene Ontology structure, such as Resnik, Lin, 
# or Jiang-Conrath measures, among others.
# Here we perform a basic example using the goatools library to calculate semantic 
# similarity with the Resnik measure and correct the list of GO IDs associated with each gene ID.

In [68]:
# since we have already goatools installed, we just check the goatools version
from importlib.metadata import version
version('goatools')


'1.3.1'

In [ ]:
# import specific tools
from goatools.semantic import semantic_similarity
from goatools import obo_parser
from goatools.obo_parser import GODag

godag = GODag("go-basic.obo")

In [69]:
# Calculate semantic similarity using Resnik measure and update the lists 
# (add updated lists to new dict, do not alter the original dict)
aspect_P_corrected = {}
for gene_id, go_ids in cleaned_data.items():
    new_go_ids = []
    for go_id in go_ids:
        max_sim = 0.0
        for other_go_id in go_ids:
            if go_id != other_go_id:
               sim = semantic_similarity(go_id, other_go_id, godag)
               max_sim = max(max_sim, sim)
        if max_sim >= 0.5:  # Set the threshold as needed
            new_go_ids.append(go_id)
    aspect_P_corrected[gene_id] = new_go_ids


In [70]:
# Print the first three key, value pairs of the corrected "P" dictionary as an example
for gene_id, go_ids in list(aspect_P_corrected.items())[:10]:
    print(f"{gene_id}: {go_ids}")

A0A075B6H7: ['GO:0002250', 'GO:0006955']
A0A075B6H8: ['GO:0002250', 'GO:0006955']
A0A075B6H9: ['GO:0002250', 'GO:0006955']
A0A075B6I0: ['GO:0002250', 'GO:0006955']
A0A075B6I1: ['GO:0002250', 'GO:0006955']
A0A075B6I3: ['GO:0002250', 'GO:0006955']
A0A075B6I4: ['GO:0002250', 'GO:0006955']
A0A075B6I6: ['GO:0002250', 'GO:0006955']
A0A075B6I7: ['GO:0002250', 'GO:0006955']
A0A075B6I9: ['GO:0002250', 'GO:0006955']


### Assessment of our method

In [71]:
# The gene codenamed A0A024RBG1 previously had 4 GO terms associated with it 
# ['GO:1901907', 'GO:1901909', 'GO:0071543', 'GO:1901911']
# Let's see how many there are now (after semantic similarity correction using Resnik measure)
aspect_P_corrected["A0A024RBG1"] 

['GO:1901907', 'GO:1901909']

In [72]:
# The gene codenamed A0A075B6H5 previously had only 1 GO term associated with it 
# ['GO:0007166']
# Let's see how many there are now (after semantic similarity correction using Resnik measure)
aspect_P_corrected["A0A075B6H5"] 

[]

In [ ]:
# So we notice another fault in our code: 
# Genes with only one term associated with them get that term deleted 
# during our similarity calculation (perhaps because there is no other term to compute similarity)

In [ ]:
# There are 4 things that we should investigate
# 1) Why did a specific term get deleted and others not?
# 2) Why in cases of only 1 term, this term gets deleted?
# 3) Why in some cases of multiple terms, all terms get deleted?
# 4) What value should the similarity threshold take?
# Extra notes
# While investigating the first 100 genes (before and after correction)
# I notice that two terms 'GO:0002250', 'GO:0006955' always survive the correction 
# and seem to almost be the only ones to do so


In [73]:
# count genes with empty values after correction

# Initialize a counter for empty values
empty_values_count = 0

# Loop through the values in the dictionary and count the empty lists
for value in aspect_P_corrected.values():
    if not value:
        empty_values_count += 1

print("Number of keys with empty values:", empty_values_count)

Number of keys with empty values: 9503


In [74]:
len(aspect_P_corrected)

17818

In [ ]:
# We see that most of the keys (genes) got all their values (GO terms) deleted after our "correction".

In [78]:
# Now let's check how many genes in the original dataset had only 1 go term associated with them

# Initialize a counter for keys with one GO term in their value list
keys_with_one_go_term_count = 0

# Loop through the values in the dictionary and count the keys with only one GO term
for value in cleaned_data.values():
    if len(value) == 1:
        keys_with_one_go_term_count += 1

print("Number of keys with only one GO term:", keys_with_one_go_term_count)

Number of keys with only one GO term: 2579


In [79]:
# Count how many different GO terms exist in the original dictionary

# Create a set to store unique GO terms
unique_go_terms = set()

# Loop through the values in the dictionary and add GO terms to the set
for value in cleaned_data.values():
    unique_go_terms.update(value)

# Get the count of different GO terms
num_unique_go_terms = len(unique_go_terms)

print("Number of different GO terms:", num_unique_go_terms)

Number of different GO terms: 12389


In [80]:
# Count how many different GO terms exist in the corrected dictionary

# Create a set to store unique GO terms
unique_go_terms = set()

# Loop through the values in the dictionary and add GO terms to the set
for value in aspect_P_corrected.values():
    unique_go_terms.update(value)

# Get the count of different GO terms
num_unique_go_terms = len(unique_go_terms)

print("Number of different GO terms:", num_unique_go_terms)

Number of different GO terms: 7240


### Other methods

In [81]:
# NEW METHOD
def get_most_general(go_terms, godag):
    most_general_term = None
    max_similarity = 0.5  # Semantic similarity threshold
    for term1 in go_terms:
        for term2 in go_terms:
            if term1 != term2:
                similarity = semantic_similarity(term1, term2, godag)
                if similarity > max_similarity:
                    most_general_term = term1 if godag[term1].level < godag[term2].level else term2
                    break
        if most_general_term is not None:
            break
    return most_general_term

# Load the GO DAG
obo_file = "go-basic.obo"  
go_dag = obo_parser.GODag(obo_file)

# Create a new dictionary with the reduced GO terms
new_data_dict = {}
for key, go_terms in cleaned_data.items():
    if len(go_terms) == 1:
        new_data_dict[key] = go_terms[0] #if there is only 1 go term, keep it (avoid empty values)
    else:
        most_general_term = get_most_general(go_terms, go_dag)
        if most_general_term is not None:
            new_data_dict[key] = most_general_term
        else:
            # If no term is more general (similarity threshold not met), keep all terms
            new_data_dict[key] = go_terms

# What if similarity threshold is met but both terms are on the same lvl (meaning that none is more 
# general than the other?)

go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms


In [83]:
for gene_id, go_ids in list(new_data_dict.items())[:10]:
    print(f"{gene_id}: {go_ids}")

A0A075B6H7: GO:0006955
A0A075B6H8: GO:0006955
A0A075B6H9: GO:0006955
A0A075B6I0: GO:0006955
A0A075B6I1: GO:0006955
A0A075B6I3: GO:0006955
A0A075B6I4: GO:0006955
A0A075B6I6: GO:0006955
A0A075B6I7: GO:0006955
A0A075B6I9: GO:0006955


In [ ]:
# check revigo for this example (key:A0A075B6H7)('GO:0002250', 'GO:0006955', 'GO:0002377') 
# (options in revigo: method = Reskind, resulting list 0.5)
# our code works because (like revigo) its keeps only the term GO:0006955 which is the most general

In [86]:
new_data_dict['A0A024RBG1']

['GO:1901907', 'GO:1901909', 'GO:0071543', 'GO:1901911']

In [87]:
cleaned_data['A0A024RBG1']

['GO:1901907', 'GO:1901909', 'GO:0071543', 'GO:1901911']

In [ ]:
# check revigo for this example ['GO:1901907', 'GO:1901909', 'GO:0071543', 'GO:1901911']
# our code keeps all the terms but revigo keeps only GO:0071543 and GO:1901911

In [ ]:
#OTHER METHOD (not working)

#go_data = obo_parser.GODag("go-basic.obo")

#def reduce_go_terms(go_terms, threshold=0.8):
#    reduced_terms = set(go_terms)
#
#    for term1 in go_terms:
#        for term2 in go_terms:
#            if term1 != term2:
#                similarity = semantic_similarity(term1, term2, go_data, ic_map)
#                if similarity >= threshold:
#                    reduced_terms.discard(term2)
#
#    # Make sure at least one GO term is always included
#    if not reduced_terms:
#        reduced_terms.add(min(go_terms, key=lambda x: ic_map[x]))
#
#    return list(reduced_terms)

# Now, if the reduction process removes all GO terms for a key, we will add back the 
# GO term with the minimum information content (ic_map[x]) to ensure that at least one
# term is retained.

#reduced_dictionary = {}

#for key, go_terms in cleaned_data.items():
#    reduced_terms = reduce_go_terms(go_terms)
#    reduced_dictionary[key] = reduced_terms

In [ ]:
# All in all i believe I am close to the solution, just need to pinpoint a couple of things:

# 1) What similarity threshold to use?
# 2) If two terms surpass the similarity threshold (meaning they are too similar) which do we keep?
# (the more general or the more specific and what if they are one the same "level")

# I believe if we answer these questions and tweak a little bit the code in line 81, we will have
# the desired result.